In [26]:
import os

In [27]:
import pandas as pd
import numpy as np

data = pd.read_csv('annotated_ryk_final.csv')
data.head()

,SentimentText,label,language,urgency
0,"@RailMinIndia My PNR is 8348062961, I am in wa...",3,0,0
1,@sureshpprabhu @RailMinIndia AC not working in...,7,0,0
2,@RailMinIndia I'm traveling to chennai by trai...,3,0,0
3,@RailMinIndia irctc is not responding at the t...,8,0,1
4,@DRMbhopal @RailMinIndia @sanjaygupta2012 @drm...,1,0,-1


In [28]:
import time, random
random.seed(time.time_ns())
d_len = len(data)
data.loc[random.randint(0, d_len - 1), "SentimentText"]

'@RailMinIndia @IRCTC_Ltd Vendor simply overcharging for à¤°à¥\x87à¤² à¤¨à¥\x80à¤°. Charging 20 rs instead of 15rs printed MRP. https://t.co/crClsrbPvB'

In [29]:
data = data[data["label"] != -1]
len(data)

2498

In [30]:
CLASSES: dict[int, str] = {
    -1: "None",
    0: "Spam/Scam/Garbage",
    1: "Non-Complaint ",
    # -- Rest are complaints --
    2: "Delay",
    3: "Hygeine (food, toilets, carriage)",
    4: "Ticket issue (cancellation, refund, seat change)",
    5: "Medical issue",
    6: "Safety issue",
    7: "Travel Experience",
    8: "Feedback / Suggestions (less critical/urgent)",
}
URGENCY: dict[int, str] = {
    -1: "None",
    0: "Low",
    1: "Pressing",
    2: "critical",
}
LANG: dict[int, str] = {
    -1: "None",
    0: "English",
    1: "Hindi (devanagari; purely)",
    2: "Hinglish (devanagari; with english in latin alphabet)",
    3: "Hinglish (devanagari; with english transliterated to hindi)",
    4: "Hinglish (latin; with hindi transliterated to english)",
}

In [7]:
counts = data.groupby("label").count().loc[:, "language"]
counts = pd.DataFrame(counts)
counts.rename(columns={"language": "count"}, inplace=True)
counts.index = [CLASSES[i] for i in list(counts.index)]
counts

,count
Spam/Scam/Garbage,276
Non-Complaint,551
Delay,219
"Hygeine (food, toilets, carriage)",174
"Ticket issue (cancellation, refund, seat change)",298
Medical issue,30
Safety issue,151
Travel Experience,344
Feedback / Suggestions (less critical/urgent),455


In [8]:
langs = data.groupby("language").count().loc[:, "label"]
langs = pd.DataFrame(langs)
langs.rename(columns={"label": "count"}, inplace=True)
langs.index = [LANG[i] for i in list(langs.index)]
langs

,count
English,2256
Hindi (devanagari; purely),94
Hinglish (devanagari; with english in latin alphabet),40
Hinglish (devanagari; with english transliterated to hindi),18
Hinglish (latin; with hindi transliterated to english),90


In [9]:
urgency = data.groupby("urgency").count().loc[:, "label"]
urgency = pd.DataFrame(urgency)
urgency.rename(columns={"label": "count"}, inplace=True)
urgency.index = [URGENCY[i] for i in list(urgency.index)]
urgency

,count
None,828
Low,773
Pressing,742
critical,155


In [31]:
eng_data = data[data["language"] == 0].copy()
print(len(eng_data))
eng_data.head()

2256


,SentimentText,label,language,urgency
0,"@RailMinIndia My PNR is 8348062961, I am in wa...",3,0,0
1,@sureshpprabhu @RailMinIndia AC not working in...,7,0,0
2,@RailMinIndia I'm traveling to chennai by trai...,3,0,0
3,@RailMinIndia irctc is not responding at the t...,8,0,1
4,@DRMbhopal @RailMinIndia @sanjaygupta2012 @drm...,1,0,-1


Removing lines with encoding errors, faulty data


In [32]:
print(f"Len before filtering: {len(eng_data)}")

print(*list(eng_data[eng_data["SentimentText"].str.count(r"\?") > 10]["SentimentText"]), sep="\n")
# Filter out encoding mistakes (as repeated question marks)
eng_data = eng_data[eng_data["SentimentText"].str.count(r"\?") <= 10]
print(f"Len after filtering: {len(eng_data)}")


Len before filtering: 2256
@RailMinIndia Sir, ?? ?????? ????? ?????? ?? ?????????? ?? 6 ?? water point ???? ???? ??? ?? ??? ???  ?????? ??????. https://t.co/sMfHUZdN2U
@RailMinIndia ????? ???? 22415 , current location ????????? , B6 coach , AC ???? ?? ???, ??? ??? ??? ????
@sureshpprabhu @RailMinIndia Enjoying WiFi at Lko station.Good speed.15% service tax ????  WIFi ?? ???? ???cleaning is also good here.
@sureshpprabhu @RailMinIndia @drmbikaner @SrdcmC  ??!STNL stn ?? ??? concerned officer ???? ???????????? ??,???? ???? mini ??? only ?? ??? ?
@RailMinIndia Sir,???? ?????? ???? Tweet Delete ?? ???? ??,water point ?? ???????? ???? ???? ?? ??? ??,???? ??????? ???? ?? ???? action ????
@sureshpprabhu  Sorry to say bitter true that there is no ?????? ???? ?????? at Patna Jn. Still dirty platform n rail tracks. @RailMinIndia
@RailMinIndia  ?????? ??? ?? ?????? ?? ??? ????? ??? ???? ?? ?? ????? ???? ?? ??????#FearOfDelayed
@RailMinIndia ?? ??xerox ???? ?? ??? ???? ??? ?? ? ?? ??? ??? ??,?????

In [33]:
print(f"Len before filtering: {len(eng_data)}")

print(*list(eng_data[eng_data["SentimentText"].str.contains(r"[µ¥\x95\x87¿¡¦à\x8c²¯¨°¤\x8b\x80\xadª£\x8f¾¹¬\x88®\x8d]{3,}")]["SentimentText"]), sep="\n")
# Filter out encoding mistakes (as repeated question marks)
eng_data = eng_data[~eng_data["SentimentText"].str.contains(r"[µ¥\x95\x87¿¡¦à\x8c²¯¨°¤\x8b\x80\xadª£\x8f¾¹¬\x88®\x8d]{3,}")]
print(f"Len after filtering: {len(eng_data)}")

Len before filtering: 2248
@RailMinIndia à¤à¤ à¤¬à¤¾à¤¢à¤¼ à¤¸à¥à¤à¥à¤¶à¤¨ à¤¸à¥ à¤®à¥à¤à¤¾à¤®à¤¾ à¤¸à¥à¤à¥à¤¶à¤¨ à¤à¥ à¤¬à¥à¤ à¤à¥à¤°à¥à¤¨ no.13226 à¤®à¥à¤ 30km à¤®à¥à¤ 60 à¤¬à¤¾à¤° à¤à¥à¤¨ à¤ªà¥à¤²à¤¿à¤à¤ à¤¹à¥à¤,à¤à¤¸à¤¾ à¤°à¥à¤ à¤¹à¥à¤¤à¤¾ à¤¹à¥ à¤à¤ à¤à¥à¤ à¤à¥à¤¯à¤¾à¤¦à¤¾ à¤¹à¥à¤
@RailMinIndia à¤¯à¥ à¤à¤ à¤¦à¤¶à¤à¥à¤ à¤¸à¥ à¤¬à¤¦à¤¹à¤¾à¤² à¤ªà¥à¤² #à¤à¤à¤à¤¾à¤°à¤ªà¥à¤° à¤­à¥ à¤à¤ª à¤²à¥à¤ à¤¦à¥à¤ à¤²à¥à¤à¤¿à¤¯à¥ à¤¯à¥ à¤­à¥ RAILWAY à¤à¤¾ à¤¹à¥ à¤¹à¥ @sureshpprabhu RT PLS. https://t.co/0d16iFGr74
@sureshpprabhu @GMSECR @RailMinIndia à¤à¤¾ à¤à¤ à¤¬à¤¹à¥à¤à¤ªà¤¯à¥à¤à¥ à¤à¤à¤¡à¥à¤°à¥à¤¯à¤¡ à¤à¤ªà¥à¤²à¥à¤à¥à¤¶à¤¨à¥¤ à¤¡à¤¾à¤à¤¨à¤²à¥à¤¡ à¤à¤°à¥à¤...https://t.co/KqdS66Nusp https://t.co/9NO9H412ZX
@RailMinIndia à¤à¥à¤°à¥à¤¨ à¤®à¥à¤ 7 à¤°à¥ à¤à¤¾ à¤à¤¾à¤¯ 10 à¤°à¥ à¤®à¥à¤, 15 à¤°à¥ à¤à¤¾ à¤ªà¤¾à¤¨à¥ 20 à¤°à¥ à¤®à¥à¤,35 à¤°à¥ à¤à¤¾ à¤ à¤à¤¡à¤¾ 40 à¤°à¥ à¤®à¥à¤ à¤à¤°à¥à¤¦à¤¨à¥

In [34]:
eng_data = eng_data.reset_index(drop=True)


<h1> Preprocessing the text data </h1>

In [11]:
# %pip install ekphrasis -q

In [14]:
import re

from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer

tokenizer = SocialTokenizer(lowercase=False)
unpacker = TextPreProcessor(
    normalize=["date"],
    annotate={"hashtag"},
    segmenter="twitter",
    unpack_hashtags=True,
    tokenizer=tokenizer.tokenize
)

/home/risb/Desktop/BTP2/venv/lib/python3.12/site-packages/ekphrasis/classes/tokenizer.py:225: FutureWarning: Possible nested set at position 2190
  self.tok = re.compile(r"({})".format("|".join(pipeline)))


Reading twitter - 1grams ...
Reading twitter - 2grams ...


/home/risb/Desktop/BTP2/venv/lib/python3.12/site-packages/ekphrasis/classes/exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


Reading english - 1grams ...


In [ ]:
# from nltk.corpus import stopwords
# from nltk.tokenize import word_tokenize
# from nltk.tokenize.treebank import TreebankWordDetokenizer
# import nltk

# nltk.download("stopwords")
# nltk.download("punkt")
# nltk.download("punkt_tab")

[nltk_data] Downloading package stopwords to /home/risb/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/risb/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/risb/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:

# stopping = set(stopwords.words("english"))
# stopping.remove("not")
# stopping.remove("no")
# stopping.remove("i")
# stopping.remove("and")
# stopping.remove("why")
# stopping.remove("do")
# stopping.remove("more")
# stopping.remove("than")
# stopping.remove("m")
# stopping.remove("can")
# stopping.remove("t")
# detok = TreebankWordDetokenizer()

In [35]:
replacements = [
    ("grp", "group"), ("rs", "rupees"), (r"hr(s)?", r"hour\1"),
    (r"pl[sz]*", "please"), ("jn", "junction"), ("nite", "night"),
    ("r", "are"), ("exp(rs)?", "express"), ("pf", "platform"),
    ("u", "you"), ("nd", "and"), ("frm", "from"), ("n", "and"),
    ("ur", "your"), (r"tc?kt(s)?", r"ticket\1"), ("stn", "station"),
    ("luks", "looks"), ("i'm", "i am"), ("haven't", "have not"),
    ("you're", "you are"), ("she's", "she is"), ("he's", "he is"),
    ("can't", "can not"), ("hvng", "having"), ("what's", "what is"),
    ("it's", "it is"), ("isn't", "is not"), ("doesn't", "does not"),
    ("ppl", "people"), ("is'nt", "is not"), ("does'nt", "does not"),
    ("smtng", "something"), ("dnt", "do not"), ("don't", "do not"),
    ("da", "the"), ("d", "the"), ("do'nt", "do not"), ("y", "why"),
    ("s", "is"), ("wen", "when"), (r"ha?ve?", "have"), ("y", "why"),
    ("haven't", "have not"), ("have'nt", "have not"), ("hv", "have"),
    ("hon'ble", "honourable"), (r"luk([s]?)", r"look\1"),
    ("b", "be"), ("bt", "but"), (r"di[sz]", "this"), ("hw", "how"),
    ("askd", "asked"), ("stns", "stations"), ("drty", "dirty"),
    ("smthng", "something"), (r"trn([s]?)", r"train\1"),
    ("sl", "sleeper"), (r"w\/l", "waiting list"), ("dint", "did not"),
    ("mrp", "retail price"), ("rac", "reserved"), (r"tt[e]?", "ticket collector"),
    (r"xing([s]?)", r"crossing\1"), ("hpnd", "happened"), ("tc", "ticket collector"),
    (r"tym([s]?)", r"time\1"), ("hpnd", "happened"), ("cnf", "confirmed"),
    ("bak", "back"), ("rpf", "railway police"), ("v", "we"), ("ac", "air conditioner"),
    ("msg", "message"), (r"tha?nx", "thanks"), ("ty", "thanks"),
    (r"wo?rki?ng", "working"), (r"yr([s]?)", r"year\1"), ("gt", "get"),
    (r"&gt;", "more than"), (r"&lt;", "less than"), (r"bc[o]?[sz]", "because"),
    ("ub", "upper berth"), ("mb", "middle berth"), ("lb", "lower berth"),
    ("boogie", "bogie"), ("boogy", "bogie"), ("boggy", "bogie"),
    (r"cha?r?ge?d", "charged"), ("rly", "railway"), (r"reqd?", "required"),
    (r"wa?i?ti?ng", "waiting"), (r"de?la?ye?d", "delayed"), ("tikit", "ticket"),
    (r"pro?pe?r", "proper"), (r"â¹([0-9]+)", r"\1 rupees"), ("tikit", "ticket"),

    ("nt", "not"), ("m", "am"), ("shd", "should"), ("ll", "will"),
    ("gv", "give"), ("fr", "for"), ("al", "all"), ("thr", "there"),
    # PNR removal
    (r"pnr *(?:no *\.? *\:?)?", " "),
    ("ws", "was"), (r"st\.", "station"), (r"no *\.? *(?=:? *[1-9][0-9]*)", "number "),
    (r"re?qu?e?st", "request"), ("sum1", "someone"),
    (r"([1-9][\-0-9]*) *(st|nd|rd|th)", r"\1\2"),
]
replacements_cs = [(i, j) for (i, j) in replacements if len(i) == 1]
replacements = [(i, j) for (i, j) in replacements if len(i) > 1]


In [19]:
len(eng_data)

2196

In [ ]:
for i in range(len(eng_data)):
    text = eng_data.loc[i, "SentimentText"]
    # print(f"Original: {text}")
    # print(f"Processed: {text}")

    # PNR number removal
    text = re.sub(r"[1-9][0-9]{5,}", "", text)

    for (before, after) in replacements_cs:
        text = re.sub(f"(?<![a-zA-Z]){before}(?!([a-zA-Z])|[0-9]+)", after, text)
    text = text.lower()

    text = re.sub(r"â¦", " ", text)
    # Remove non-functional hashtags
    text = re.sub(r"#(\s)", r"\1", text)
    # Removing mentions
    text = re.sub(r'@[\w]+', '', text)
    text = re.sub(r'\s?@\s?', ' at ', text)
    # Remove old retweet format
    text = re.sub(r'rt\s*:\s*', '', text)
    # Renaming 'and'
    text = re.sub(" ?&amp; ?", " and ", text)
    # Removing links
    text = re.sub(r"https?://\S+", "", text)
    # Replace repeating special chars w/ only one
    text = re.sub(r'([^\w\s])\1+', r'\1', text)
    # Encoding issue fix for rupee symbol
    text = re.sub(r"\?([0-9]+)", r"\1 rupees", text)

    for (before, after) in replacements:
        text = re.sub(f"(?<![a-zA-Z]){before}(?![a-zA-Z])", after, text)

    words = [
        i for i in unpacker.pre_process_doc(text) if i not in (
            "<hashtag>", "</hashtag>", "#"
        )
    ]
    text = " ".join(words)

    text = re.sub(r"([1-9][\-0-9]*) *(st|nd|rd|th)", r"\1\2", text)



    # Join apostrophy to contractions
    text = re.sub(f"(?<=[a-zA-Z]) ' (?=[a-zA-Z])", "'", text)

    # stop word removal
    # words = word_tokenize(text)
    # filtered = [word for word in words if word.lower() not in stopping]
    # text = detok.detokenize(filtered)

    # print(text + "\n")
    eng_data.loc[i, "SentimentText"] = text
    # if i == 100: break

In [24]:
len(eng_data)

2196

Segment Dataset based on First model (Complaint/Non-Complaint)



In [ ]:
def generalize_label(label):
    return 0 if label <= 1 or label == 8 else 1

CNC_CLASSES = {
    0: "Non-Complaint",
    1: "Complaint",
}

eng_data_cnc = eng_data.copy()
eng_data_cnc["label"] = eng_data_cnc["label"].map(generalize_label)
eng_data_cnc = eng_data_cnc.drop(['language', 'urgency'], axis=1)
# eng_data_cnc.head()

eng_data_cnc.to_csv("eng_data_cnc.csv", index=False)

In [18]:
eng_data_c_urg = eng_data.copy()
eng_data_c_urg = eng_data_c_urg[eng_data_c_urg["urgency"] != -1]
eng_data_c_urg = eng_data_c_urg.drop(['label', 'language'], axis=1)
eng_data_c_urg.to_csv("eng_data_c_urg.csv", index=False)

In [19]:
eng_data_c_type = eng_data.copy()
eng_data_c_type = eng_data_c_type.drop(['language', 'urgency'], axis=1)


In [20]:
def club_label(label):
    if label in (0, 8):
        return 1
    return label
def decr_one(label):
    return label - 1
eng_data_c_type["label"] = eng_data_c_type["label"].map(club_label)
eng_data_c_type["label"] = eng_data_c_type["label"].map(decr_one)

In [22]:
eng_data_c_type.to_csv("eng_data_c_type.csv", index=False)